## Build model using data from file

We use the `pandas` module to read in a large datafile that we then use to create a problem. The datafile contains the edges of a directed graph $G=(V,A)$ and we will use the data to create a shortest-path problem from the first to the last node (sorted by their index). The format of the data file is: `i,j,c`, where $i$ and $j$ are the endnodes of the directed $i\to j$ arc, while $c$ is the cost of the arc.

In [14]:
import pandas as pd
import xpress as xp

# Read graph data
df = pd.read_csv('graph_data.csv')

# Isolate tail/head of each arc and its cost in three separate vectors
tail = df['i']
head = df['j']
cost = df['c']

V = set(tail) | set(head)  # get set V of nodes

# Create A as a dictionary with cost as value
A = {(tail[i],head[i]) : cost[i] for i,_ in enumerate(tail)}

source = min(V)
dest = max(V)

In [20]:
# Create one variable for each arc
# x= {(i,j): xp.var(vartype=xp.binary) for i in range(V) for j in range(V)}
x = xp.vars(list(A.keys()), vartype=xp.binary)

# Define flow conservation constraint: first, the right-hand side.
b = {i: 0 for i in V}

b[source] =1
b[dest] =-1

# Second, the conservation constraint:
#   the total outgoing flow from node i
#   minus the total incoming flow into node i
#   must equal 0 in all intermediate node
#   and 1, -1 at source and destination respectively.
conservation =[
    xp.Sum(x[(i,j)] for j in V if (i,j) in A.keys()) -
    xp.Sum(x[(j,i)] for j in V if (j,i) in A.keys()) == b[i] for i in V
]

# Define the objective: the sum over all costs times all arc variables
obj = xp.Sum(A[(i,j)] * x[(i,j)] for (i,j) in A)

p = xp.problem(x, conservation, obj)
p.solve()

# Print all arcs that constitute a solution, i.e. those with value of x at 1.
for (i,j) in A:
   if p.getSolution(x[(i,j)]) > 0.5:
       print(i,j)


FICO Xpress v8.9.0, Hyper, solve started 11:26:14, Sep 20, 2020
Heap usage: 334KB (peak 334KB, 582KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing MILP noname
Original problem has:
         5 rows           19 cols           38 elements        19 globals
Presolved problem has:
         5 rows           19 cols           38 elements        19 globals
Presolve finished in 0 seconds
Heap usage: 363KB (peak 363KB, 584KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 2.00e+00,  1.90e+01] / [ 2.00e+00,  1.90e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 1.3GB
 *** Heuristic solution found:    19.000000      Time: 0 ***
 *** Heuristic solution found:

## Indicator constraints

Indicator constraints are constraints that must hold if a given binary variable is equal to one, and they are not enforced otherwise. For instance, if a problem has two variables $x$ and $y$ and we want to constrain them to $x+2y \ge 4$ only if a third, binary variable $z$ is set to one, then this can be denoted as 

$$
z=1 \Rightarrow x + 2y \ge 4
$$

Indicator constraints are common in Mixed Integer Linear Optimization. In the Xpress Python interface, the above indicator constraint would be added to a problem as follows:

```
p.addIndicator (z == 1, x + 2*y >= 4)
```

## General functions

The Xpress Optimizer allows for modeling specific non-linear functions so that they are automatically reformulated within a Mixed Integer format. These functions are:

* The absolute value of a function, $|x|$;
* The maximum and minimum of a list of functions, e.g. $\max(x+1, 2y-2,...)$;
* The logical And and Or operators on a set of functions, for example $(x \wedge y) \vee z$; these operators work on binary variables.

While the C interface of the Xpress optimizer uses low-level functions for this, the Python interface allows for natural modeling using the operators `xpress.abs`, `xpress.max`, `xpress.min`, `xpress.And`, `xpress.Or`, and the binary Python operators `&` and `|`. Let's see an example of this.

In [21]:
import xpress as xp

x = xp.var()
y = xp.var()

# Create a problem whose objective function is
#
# |x-y| + 2 max(2x+4, 3y+2x)
obj= xp.abs(x-y) + 2*xp.max(2*x+4, 3*y+2*x)

# and with constraints
# x+2y >= 20 and x+3y <= 25
c1= x+2*y>=20
c2= x+3*y<=25

p = xp.problem(x,y,c1,c2,obj)

p.solve()

print(p.getSolution(x), p.getSolution(y))


FICO Xpress v8.9.0, Hyper, solve started 11:26:40, Sep 20, 2020
Knitro not found, link disabled

Original problem size
   linear:    3 rows, 4 columns, 5 linear coefficients
   nonlinear: 1 coefficients, 23 tokens
Nonlinear presolve
   compressing formula space (in use\total : 24\49)
   removed 2 formulas and 24 formula tokens
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 1 ABS to optimizer MIP constructs
   converted 1 MIN/MAX to optimizer MIP constructs
   converted 4 formulas to linear constraints
   converted objective transfer row to linear objective
   problem is equivalent, reformulated as a MIP problem
Minimizing problem using Xpress-Optimizer
FICO Xpress v8.9.0, Hyper, solve started 11:26:40, Sep 20, 2020
Heap usage: 680KB (peak 810KB, 902KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing MILP noname
Original problem has:
         6 rows            9 cols       

### Manual MIP reformulation of $|x|$, $\max/\min$, and AND/OR

In the above problem, we defined a quantity $|x-y|$ with the `xpress.abs` operator.

Below is how we would have to model the constraint $z = |x-y|$ if we did not have the `xp.abs` function. We would have to model $z = |x-y|$ by introducing a binary variable $w$ and writing the following constraints, two of which are indicator constraints:

$$
\begin{array}{lll}
z \ge x-y\\
z \ge y-x\\
w = 1 \Rightarrow z \le x-y\\
w = 0 \Rightarrow z \le y-x\\
w \in \{0,1\}
\end{array}
$$

If, however, we only had the *inequality* constraint $z \ge |x-y|$, then reformulation would be a lot easier: the two constraints

$$
z \ge x - y; \qquad z \ge y-x
$$

would suffice.

And/Or operator do not require extra variable: for an operation $z = x \wedge y$, with $x$ and $y$ binary variables, one can write

$$
\begin{array}{l}
z \le x\\
z \le y\\
z \ge x + y - 1
\end{array}
$$

and similar for an operation $z = x \vee y$.

The Xpress Python interface allows for writing

* `xpress.abs(2*x-4)` to create the expression $|2x-4|$;
* `xpress.min(x1, x2, 2*x3-1)` for the expression $\min(x_1, x_2, 2x_3-1)$, and similar for the maximum;
* `xpress.And(y1, y2, y3, 1-y4)` for the logical And between the binary variables $y_1$, $y_2$, $y_3$, and the negation of $y_4$, which can be modeled as $1-y_4$ as it is 0 when $y_4=1$ and viceversa, functioning therefore as a negation operator. 

In [24]:
import xpress as xp

# Create two binary variables x and y and then a list of 10 binary
x = xp.var(vartype=xp.binary)
y = xp.var(vartype=xp.binary)
z = xp.vars(10, vartype=xp.binary)

# Create a problem whose objective function is the sum of all z variables
# and with constraints
#
# (x AND y) + (x OR y OR z[0] OR z[1] OR z[4]) == 1
constr = (x&y) + (x | y | z[0] | z[1] | z[4]) ==1
obj = xp.Sum(z)
p = xp.problem(x,y,z,constr, obj)
p.solve()


FICO Xpress v8.9.0, Hyper, solve started 11:28:34, Sep 20, 2020
Original problem size
   linear:    1 rows, 13 columns, 0 linear coefficients
   nonlinear: 1 coefficients, 18 tokens
Nonlinear presolve
   compressing formula space (in use\total : 19\39)
   removed 2 formulas and 19 formula tokens
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 5 MIN/MAX to optimizer MIP constructs
   converted 1 formulas to linear constraints
   problem is equivalent, reformulated as a MIP problem
Minimizing problem using Xpress-Optimizer
FICO Xpress v8.9.0, Hyper, solve started 11:28:34, Sep 20, 2020
Heap usage: 678KB (peak 807KB, 898KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing MILP noname
Original problem has:
         1 rows           18 cols            2 elements        12 globals
         5 gencons
Presolved problem has:
         9 rows            8 cols           22 elements    

**Exercise**: Formulate a MIP to solve a SATisfaction problem defined by the clauses on three variables as defined below, where $x_i$ means a variable is included as-is and $\bar x_i$ means its negation:

$$
\begin{array}{lllll}
  \bar x_1 &\vee&      x_2 &    &          \\
           &    & \bar x_2 &\vee&      x_3 \\
  \bar x_1 &    &          &\vee&      x_3 \\
           &    & \bar x_2 &\vee& \bar x_3 \\
       x_1 &    &          &\vee&      x_3 \\
       x_1 &\vee& \bar x_2 &    &          \\
\end{array}
$$

In [25]:
# Identify each clause with -1 if variable is negated, 1 if non-negated, 0 if not present.

id_clauses = [
    [-1, 1, 0],
    [0, -1, 1],
    [-1, 0, 1],
    [0, -1, -1],
    [1, 0, 1],
    [1, -1, 0]]

I = [0,1,2]

# TODO Create a list of three binary variables
x = [xp.var(vartype=xp.binary) for _ in I]

# TODO Create a list of clauses identified, in turn, as lists of three elements
# Containing x[0] or 1-x[0] depending on whether the variable is negated
# in id_clauses.
# a= xp.Or(*id_clauses)

# TODO Create a list of constraints where the logical OR of the members 
# of a clause must be 1, as per definition of SAT.

# TODO Create a problem. Note that we don't need an objective function

# p.solve()
# print(p.getSolution(x))

ModelError: Invalid object in n-ary operation

## Simple cutting plane algorithm

We'll implement a very simple cutting plane algorithm. Consider the problem

$$
\begin{array}{ll}
\min &\sum_{i=1}^n\sum_{j=1}^n q_{ij} x_i x_j + \sum_i c_i x_i\\
\textrm{s.t.} & x_i\in \{0,1\} & \forall i=1,2,\ldots, n.
\end{array}
$$

This problem is a Binary Box-constrained Quadratic Programming problem: it has a quadratic objective function and all variables are negative; furthermore, the only constraints are the implicit bounds on binary variables, i.e. $\{0,1\}$. The problem can be written in matricial form as follows:

$$
\begin{array}{ll}
\min & x^T Q x + c^T x\\
\textrm{s.t.} & x \in \{0,1\}^n
\end{array}
$$

where $Q$ is a symmetric matrix. In general, the above problem is much harder if the term $x^T Q x$ is nonconvex, i.e., if $Q$ is **not** positive semidefinite. Let's try an example.

In [31]:
import xpress as xp
import numpy as np

np.random.seed(1234567)

n = 30
Q = np.random.random((n,n)) - .5
c = np.random.random(n) - .5

# Create a numpy array of n binary variables
x= xp.vars(n, vartype=xp.binary)

# Create a Binary Quadratic problem with aforementioned objective function.
obj=xp.Sum(x.T*Q*x+c.T*x)
p=xp.problem(x,obj)
p.solve()



FICO Xpress v8.9.0, Hyper, solve started 11:34:37, Sep 20, 2020
Heap usage: 335KB (peak 335KB, 607KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing MIQP noname
Original problem has:
         0 rows           30 cols            0 elements        30 globals
        30 qobjelem
Presolved problem has:
         0 rows           30 cols            0 elements        30 globals
Presolve finished in 0 seconds
Heap usage: 343KB (peak 364KB, 609KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [      0.0,       0.0] / [      0.0,       0.0]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.30e-01,  1.44e+01] / [ 2.01e-01,  1.56e+01]
  Quadratic      [min,max] : [ 4.78e-01,  8.59e+00] / [      0.0,       0.0]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 

In [32]:
p.getSolution()

[-0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 1.0,
 1.0,
 -0.0,
 1.0,
 -0.0]

The Xpress Optimizer can solve such problems after a *reformulation* that does the following:

* Creates a (binary) variable $y_{ij}$ for each product $x_i x_j$;
* For each variable $y_{ij}$, it adds the following constraints linking $y$ with $x$ variables:

$$
\begin{array}{lll}
y_{ij} &\le& x_i \\
y_{ij} &\le& x_j \\
y_{ij} &\ge& x_i + x_j - 1
\end{array}
$$

After this reformulation, the problem becomes as follows:

$$
\begin{array}{ll}
\min & \sum_{i=1}^n \sum_{j=1}^n q_{ij} y_{ij} + \sum_{i=1}^n c_i x_i\\
\textrm{s.t.} &y_{ij} \le x_i  &\forall i,j\\
&y_{ij} \le x_j  &\forall i,j\\
&y_{ij} \ge x_i + x_j - 1 &\forall i,j\\
&x_i \in \{0,1\} \forall i=1,2,\ldots,n\\
&y_{ij} \in \{0,1\} \forall i,j=1,2,\ldots,n\\
\end{array}
$$

We can do this reformulation explicitly by introducing the variables $y_{ij}$. Below we use NumPy vectors of variables as they come in handy to write constraints.

In [42]:
import xpress as xp
import numpy as np

np.random.seed(1234567)

n = 30
Q = np.random.random((n,n)) - 0.5
c = np.random.random(n) - 0.5

# Create variables x AND a matrix (i.e. two-index numpy array)
# of variables, called Y.
x = xp.vars(n, vartype=xp.binary)
Y = xp.vars(n, n, vartype=xp.binary)


# Create a problem with x and Y and linearized objective function,
# plus obviously the linearization constraints above.
p = xp.problem(
    x, Y,
    xp.Sum(x.T*Q*x+c.T*x),
    Y<=x,
    Y<=x.T,
    Y<=x + x.T -1
)

p.solve()
p.getSolution()


FICO Xpress v8.9.0, Hyper, solve started 11:49:40, Sep 20, 2020
Heap usage: 1249KB (peak 1249KB, 1555KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing MIQP noname
Original problem has:
      2700 rows          930 cols         5400 elements       930 globals
        30 qobjelem
Presolved problem has:
         0 rows            0 cols            0 elements         0 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 1184KB (peak 2410KB, 1557KB system)
Will try to keep branch and bound tree memory usage below 1.3GB
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0         35.137271      D      0     0        .000000     0
Optimal solution found
Dual solved problem
  0 simplex iterations in 0s

Final objective                       : 3.513727134892899e+01
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /      

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 

And this is where we start from with our cutting plane algorithm. The following are well-known *valid inequalities* for the above problem, each holding for each triple of distinct indices $(i,j,k)$ where $i<j<k$:

$$
\begin{array}{l}
y_{ij} + y_{jk} + y_{ik} + 1 \ge x_i + x_j + x_k\\
-y_{ij} + y_{jk} - y_{ik} + x_i \ge 0\\
-y_{ij} - y_{jk} + y_{ik} + x_j \ge 0\\
 y_{ij} - y_{jk} - y_{ik} + x_k \ge 0
\end{array}
$$

For any value of $n$, the number of variables, there are about $n^3$ such inequalities. Adding all of them to the original model would no doubt improve the lower bound, but would also make the problem much larger.

Instead we add them iteratively, following the (**cutting plane**) algorithm below:

1. Solve the LP relaxation of the problem, i.e., relax integrality constraints on $x$ and $y$ and solve the resulting continuous problem;
2. Let's denote the optimal solution of the LP problem as $(\bar {\bf x}, \bar{\bf y})$, whose components are $\bar x_i$ for $i=1,2,\ldots, n$ and and $\bar y_{ij}$ for any $i,j=1,2,\ldots, n$. The solution can be obtained with the function `p.getSolution()`.
3. Check, for all triples $(i,j,k)$, if any of the four valid inequality is violated by $(\bar {\bf x}, \bar{\bf y})$, i.e., check if
$$
\bar y_{ij} + \bar y_{jk} + \bar y_{ik} + 1 < \bar x_i + \bar x_j + \bar x_k
$$
and similar for the remaining three inequalities. If an inequality is violated, add it to the problem as a constraint.
4. If no such inequalities are violated, **stop**. Otherwise, restart from 1.

Define first a *master problem* containing the initial formulation.

In [ ]:
import xpress as xp
import numpy as np

np.random.seed(1234567)

n = 30
Q = np.random.random((n,n)) - 0.5
c = np.random.random(n) - 0.5

# TODO create continuous variables x and Y with bound interval [0,1]
x = xp.vars(n, lb=0, ub=1)
Y = xp.vars(n, n, lb=0, ub=1)

# TODO Create master problem with initial constraints as defined in previous exercise.
# p = xp.problem(
#     x, Y,
#     xp.Sum(x.T*Q*x+c.T*x),
#     Y<=x,
#     Y<=x.T,
#     Y<=x + x.T -1
# )

p = xp.problem(x, Y, xp.Sum(Q*Y) + xp.Dot(c,x),
               Y == Y.transpose(),
               Y <= x,
               Y.transpose() <= x,
               [Y[i,j] >= x[i] + x[j] - 1 for i in range(n) for j in range(n)])


# Set up cutting plane algorithm:
n_violated = -1           # Count violated inequalities. Exit the loop if none is violated
eps = 0.001               # Minimum violation for inequality to be entered
p.controls.outputlog = 0  # Silence optimizer output
MAXINEQ = 4*n             # Do not add more than this inequalities at each iteration

# STUDENT EXERCISE (to be completed)

while n_violated != 0:
    # TODO Solve LP
    p.solve()
    print("solving")
    print(p.getSolution())
    print('inequalities: {0:5d}; lower bound: {1:10.3f}'.format(n_violated, p.getObjVal()))

    n_violated = 0
    
    # Store triples defining each inequality in four separate list, one per class of inequality.
    ineq1_triples = []
    ineq2_triples = []
    ineq3_triples = []
    ineq4_triples = []
    
    # Useful to see evolution of x values from all 0.5 to different values
    # print(p.getSolution([x[i] for i in range(n)]))
    
    for i in range(n-2):
        if n_violated >= MAXINEQ:
            break
        for j in range(i+1,n-1):
            if n_violated >= MAXINEQ:
                break
            for k in range(j+1,n):
                # TODO obtain value of necessary variables using p.getSolution(x[i]), 
                # etc. (recall you need to get the Y variables too for this triple (i,j,k))
                xibar  = p.getSolution(x[i])
                xjbar  = p.getSolution(x[j])
                xkbar  = p.getSolution(x[k])
                yijbar = p.getSolution(Y[i,j])
                yjkbar = p.getSolution(Y[j,k])
                yikbar = p.getSolution(Y[i,k])

                if yijbar + yjkbar + yikbar + 1 >= xibar + xjbar + xkbar + eps:
                    n_violated += 1
                    # if so, add the corresponding triple to the first list
                    ineq1_triples.append((i,j,k))

                if  -yijbar + yjkbar -yikbar + xibar >= 0 + eps:
                    n_violated += 1
                    ineq2_triples.append((i,j,k))

                if  -yijbar - yjkbar +yikbar + xjbar >= 0 + eps:
                    n_violated += 1
                    ineq3_triples.append((i,j,k))
                    
                if  yijbar - yjkbar -yikbar + xkbar >= 0 + eps:
                    n_violated += 1
                    ineq4_triples.append((i,j,k))

                if n_violated >= MAXINEQ:
                    break
                    
    if n_violated > 0:
        for i,j,k in ineq1_triples:
            p.addConstraint(Y[i,j] + Y[j,k] + Y[i,k] + 1 >= x[i] + x[j] + x[k])
        for i,j,k in ineq2_triples:
            p.addConstraint(-Y[i,j] + Y[j,k] - Y[i,k] + x[i] >= 0)
        for i,j,k in ineq3_triples:
            p.addConstraint(-Y[i,j] - Y[j,k] + Y[i,k] + x[j] >= 0)
        for i,j,k in ineq4_triples:
            p.addConstraint(Y[i,j] - Y[j,k] - Y[i,k] + x[k] >= 0)

        # Add violated class-1 inequalities as stored in the first list of triples
#         for ii,jj,kk in ineq1_triples:
            # TODO use p.addConstraint() to add first
            # inequality based on this value of ii,jj,kk
        # TODO Do the same with the other three lists so
        # that all types of inequalities are considered.
        

print('Finished. Lower bound:', p.getObjVal())


solving
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 

## Nonlinear optimization (trigonometric, polynomial)

**Homework**. Create a MIP with nonlinear constraint and solve it using the Xpress solver using the general method `p.solve()`. Minimize a variant of the well-known Rosenbrock's *banana* function

$$
f(x,y) = (a-x)^2 + b(y-x^2)^2
$$

where $a=\frac{5}{2}$ and $b=3$, and where both $x$ and $y$ are integer.

Can you find an optimum for $a=4$ without solving the problem?

In [34]:
import xpress as xp

# TODO Create variables

a = 5/2
b = 3

# TODO Create problem (very simple: just add x, y and the objective)
x = xp.var('x',vartype=xp.integer)
y = xp.var('y',vartype=xp.integer)
con1 = (a-x)**2+b*(y-x**2)**2

# obj = x+y
pn = xp.problem(x,y,con1)
pn.solve()
pn.getSolution()


FICO Xpress v8.9.0, Hyper, solve started 17:06:37, Sep 16, 2020
Original problem size
   linear:    1 rows, 4 columns, 1 linear coefficients
   quadaric:  1 in obj, 0 rows, 0 in rows
   nonlinear: 1 coefficients, 15 tokens
Nonlinear presolve
Eliminations are disabled due to direct XPRS quadratic expressions
   objective transfer row objtransferrow through variable objtransfercol
Presolved problem size
   linear:    1 rows, 4 columns, 1 linear coefficients
   quadaric:  1 in obj, 0 rows, 0 in rows
   nonlinear: 1 coefficients, 15 tokens
Problem is nonlinear presolved
Maximum expanded nl-formula size: 15  (row 'objtransferrow')
Total tokens: 15
Quadratic formula size: 1
  32  parallel calculation threads
  Jacobian: symbolic differentiation
           1 base AD formula, 16 average complexity
           2 in the Jacobian, 20 average complexity
Initial point is within initial feasibility target with objective:  30006.250
Absolute / relative validation:            .000  /      .000
Minimiz

[2.0, 7.0, 1.0, 27.0]

In [33]:
import xpress as xp

# TODO Create variables

a = 4
b = 3

# TODO Create problem (very simple: just add x, y and the objective)
x = xp.var('x',vartype=xp.integer)
y = xp.var('y',vartype=xp.integer)
con1 = (a-x)**2+b*(y-x**2)**2

# obj = x+y
pn = xp.problem(x,y,con1)
pn.solve()
pn.getSolution()


FICO Xpress v8.9.0, Hyper, solve started 17:05:04, Sep 16, 2020
Original problem size
   linear:    1 rows, 4 columns, 1 linear coefficients
   quadaric:  1 in obj, 0 rows, 0 in rows
   nonlinear: 1 coefficients, 15 tokens
Nonlinear presolve
Eliminations are disabled due to direct XPRS quadratic expressions
   objective transfer row objtransferrow through variable objtransfercol
Presolved problem size
   linear:    1 rows, 4 columns, 1 linear coefficients
   quadaric:  1 in obj, 0 rows, 0 in rows
   nonlinear: 1 coefficients, 15 tokens
Problem is nonlinear presolved
Maximum expanded nl-formula size: 15  (row 'objtransferrow')
Total tokens: 15
Quadratic formula size: 1
  32  parallel calculation threads
  Jacobian: symbolic differentiation
           1 base AD formula, 16 average complexity
           2 in the Jacobian, 20 average complexity
Initial point is within initial feasibility target with objective:  30016.000
Absolute / relative validation:            .000  /      .000
Minimiz

[5.0, 25.0, 1.0, 0.0]

In [26]:
pn.getSolution()

[5.0, 25.0, 1.0, 0.0]

In [36]:
# The optimum for a=4 is x = 4 and y = 16, for which the 
# objective function is zero as both components of the
# objective are nonnegative.